In [1]:
import torch
# print(torch.backends.mps.is_built())

In [2]:
torch.cuda.is_available()

True

In [3]:
torch.cuda.get_device_name(0)

'NVIDIA A10G'

In [4]:
torch.cuda.mem_get_info()

(20704591872, 23689494528)

In [5]:
import pandas as pd
import os
from sklearn.preprocessing import StandardScaler
from utils.timefeatures import time_features

In [7]:
timeenc = 1
features = 'M'
scaler = StandardScaler()
root_path = '.'
data_path = 'dataset/ETT-small/ETTh1.csv'
seq_len = 96
scale = True
target = 'OT'

df_raw = pd.read_csv(os.path.join(root_path,data_path))

border1s = [0, 12 * 30 * 24 - seq_len, 12 * 30 * 24 + 4 * 30 * 24 - seq_len]
border2s = [12 * 30 * 24, 12 * 30 * 24 + 4 * 30 * 24, 12 * 30 * 24 + 8 * 30 * 24]
border1 = border1s[0]
border2 = border2s[0]
if features == 'M' or features == 'MS':
    cols_data = df_raw.columns[1:]
    df_data = df_raw[cols_data]
elif features == 'S':
    df_data = df_raw[[target]]
if scale:
    train_data = df_data[border1s[0]:border2s[0]]
    scaler.fit(train_data.values)
    data = scaler.transform(df_data.values)
else:
    data = df_data.values
df_stamp = df_raw[['date']][border1:border2]
df_stamp['date'] = pd.to_datetime(df_stamp.date)
if timeenc == 0:
    df_stamp['month'] = df_stamp.date.apply(lambda row: row.month, 1)
    df_stamp['day'] = df_stamp.date.apply(lambda row: row.day, 1)
    df_stamp['weekday'] = df_stamp.date.apply(lambda row: row.weekday(), 1)
    df_stamp['hour'] = df_stamp.date.apply(lambda row: row.hour, 1)
    data_stamp = df_stamp.drop(['date'], 1).values
elif timeenc == 1:
    data_stamp = time_features(pd.to_datetime(df_stamp['date'].values), freq='h')
    data_stamp = data_stamp.transpose(1, 0)
data_x = data[border1:border2]
data_y = data[border1:border2]
data_stamp = data_stamp

In [8]:
df_raw

,date,HUFL,HULL,MUFL,MULL,LUFL,LULL,OT
0,2016-07-01 00:00:00,5.827,2.009,1.599,0.462,4.203,1.340,30.531000
1,2016-07-01 01:00:00,5.693,2.076,1.492,0.426,4.142,1.371,27.787001
2,2016-07-01 02:00:00,5.157,1.741,1.279,0.355,3.777,1.218,27.787001
3,2016-07-01 03:00:00,5.090,1.942,1.279,0.391,3.807,1.279,25.044001
4,2016-07-01 04:00:00,5.358,1.942,1.492,0.462,3.868,1.279,21.948000
...,...,...,...,...,...,...,...,...
17415,2018-06-26 15:00:00,-1.674,3.550,-5.615,2.132,3.472,1.523,10.904000
17416,2018-06-26 16:00:00,-5.492,4.287,-9.132,2.274,3.533,1.675,11.044000
17417,2018-06-26 17:00:00,2.813,3.818,-0.817,2.097,3.716,1.523,10.271000
17418,2018-06-26 18:00:00,9.243,3.818,5.472,2.097,3.655,1.432,9.778000


In [23]:
data_stamp

array([[-0.5       ,  0.16666667, -0.5       , -0.00136986],
       [-0.45652174,  0.16666667, -0.5       , -0.00136986],
       [-0.41304348,  0.16666667, -0.5       , -0.00136986],
       ...,
       [ 0.41304348,  0.5       ,  0.3       , -0.02054795],
       [ 0.45652174,  0.5       ,  0.3       , -0.02054795],
       [ 0.5       ,  0.5       ,  0.3       , -0.02054795]])

In [10]:
data_x.shape

(8640, 7)

In [11]:
data_y.shape

(8640, 7)

In [12]:
border2s

[8640, 11520, 14400]

In [13]:
border1s

[0, 8544, 11424]

In [14]:
# at this step x and y are the same

In [15]:
data_x

array([[-0.36312285, -0.0057598 , -0.63071223, ...,  1.38857471,
         0.87514257,  1.46055158],
       [-0.38617567,  0.02629596, -0.65010053, ...,  1.32897674,
         0.92433047,  1.16152666],
       [-0.47838673, -0.13398305, -0.68869591, ...,  0.97236495,
         0.68156452,  1.16152666],
       ...,
       [ 0.07470776, -0.71050923,  0.07107161, ...,  0.49655723,
        -2.31413576,  0.42562442],
       [ 0.17844536, -0.61434181,  0.21276925, ...,  0.31776309,
        -2.16974552,  0.47161139],
       [ 0.35134102, -0.45406273,  0.27727609, ..., -0.00953795,
        -2.26653455,  0.39489377]])

In [16]:
data_y

array([[-0.36312285, -0.0057598 , -0.63071223, ...,  1.38857471,
         0.87514257,  1.46055158],
       [-0.38617567,  0.02629596, -0.65010053, ...,  1.32897674,
         0.92433047,  1.16152666],
       [-0.47838673, -0.13398305, -0.68869591, ...,  0.97236495,
         0.68156452,  1.16152666],
       ...,
       [ 0.07470776, -0.71050923,  0.07107161, ...,  0.49655723,
        -2.31413576,  0.42562442],
       [ 0.17844536, -0.61434181,  0.21276925, ...,  0.31776309,
        -2.16974552,  0.47161139],
       [ 0.35134102, -0.45406273,  0.27727609, ..., -0.00953795,
        -2.26653455,  0.39489377]])

In [17]:
data_stamp.shape



(8640, 4)

In [18]:
index = 1
label_len = 48
pred_len = 96

s_begin = index
s_end = s_begin + seq_len
print(s_begin, s_end)
r_begin = s_end - label_len
r_end = r_begin + label_len + pred_len
print(r_begin, r_end)

seq_x = data_x[s_begin:s_end]
seq_y = data_y[r_begin:r_end]
seq_x_mark = data_stamp[s_begin:s_end]
seq_y_mark = data_stamp[r_begin:r_end]

1 97
49 193


In [19]:
seq_x.shape

(96, 7)

In [20]:
seq_y.shape

(144, 7)

In [21]:
seq_x_mark.shape

(96, 4)

In [22]:
seq_y_mark.shape

(144, 4)